In [1]:
import os
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np
import pickle
import faiss
from langchain_community.vectorstores import FAISS
from langchain.retrievers.ensemble import EnsembleRetriever
from langchain.retrievers import BM25Retriever
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.vectorstores import FAISS

c:\Users\My Computer\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
JSON_DIR = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_json"
MODEL_NAME = "sberbank-ai/sbert_large_nlu_ru"
INDEX_NAME = "index"
VECTOR_STORE_PATH = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\vector_store"
K = 10

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name=MODEL_NAME)

docs = []

for filename in tqdm(os.listdir(JSON_DIR)):
    with open(os.path.join(JSON_DIR, filename), "r", encoding="utf-8") as f:
        data = json.load(f)

    doc_name = data.get("документ", filename)

    for глава in data.get("главы", []):
        глава_номер = глава.get("глава", "")
        заголовок = глава.get("заголовок", "")

        for пункт in глава.get("пункты", []):
            номер_пункта = пункт.get("номер", "")
            текст = пункт.get("текст", "")

            if not текст.strip():
                continue

            metadata = {
                "документ": doc_name,
                "глава": глава_номер,
                "заголовок": заголовок,
                "пункт": номер_пункта,
            }

            docs.append(Document(page_content=текст, metadata=metadata))

print(f"Векторизация {len(docs)} чанков и сохранение векторного хранилища...")
faiss_index = FAISS.from_documents(docs, embedding_model)

faiss_index.save_local(folder_path=VECTOR_STORE_PATH, index_name=INDEX_NAME)

print(f"\nВекторный индекс и метаданные сохранены в: {VECTOR_STORE_PATH}")
print(f"  - {INDEX_NAME}.faiss")
print(f"  - {INDEX_NAME}.pkl")

C:\Users\My Computer\AppData\Local\Temp\ipykernel_11204\1700765211.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=MODEL_NAME)
100%|██████████| 8/8 [00:00<00:00, 1133.40it/s]


Векторизация 738 чанков и сохранение векторного хранилища...

✅ Векторный индекс и метаданные сохранены в: C:\Users\My Computer\Desktop\Work\Learn\OPK_Project\vector_store
  - index.faiss
  - index.pkl


In [3]:
embedding_model = HuggingFaceEmbeddings(model_name=MODEL_NAME)
faiss_index = FAISS.load_local(VECTOR_STORE_PATH, embedding_model, index_name=INDEX_NAME, allow_dangerous_deserialization=True)

bm25_documents = []

for filename in os.listdir(JSON_DIR):
    if filename.endswith(".json"):
        path = os.path.join(JSON_DIR, filename)
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

        doc_name = data.get("документ", filename)

        for глава in data.get("главы", []):
            номер_главы = глава.get("глава", "")
            заголовок = глава.get("заголовок", "")

            for пункт in глава.get("пункты", []):
                номер_пункта = пункт.get("номер", "")
                текст = пункт.get("текст", "").strip()

                if not текст:
                    continue

                bm25_documents.append(Document(
                    page_content=текст,
                    metadata={
                        "документ": doc_name,
                        "глава": номер_главы,
                        "заголовок": заголовок,
                        "пункт": номер_пункта
                    }
                ))

bm25_retriever = BM25Retriever.from_documents(bm25_documents)
bm25_retriever.k = K

faiss_retriever = faiss_index.as_retriever(search_kwargs={"k": K})

retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5, 0.5]
)

def retrieve(query: str, k: int = K):
    results = retriever.get_relevant_documents(query)
    return results[:k]

C:\Users\My Computer\AppData\Local\Temp\ipykernel_3724\3878580727.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=MODEL_NAME)


In [5]:
results = retrieve("правила визуального полета", k=10)
for doc in results:
    print("Документ:", doc.metadata["документ"])
    print("Глава:", doc.metadata["глава"], "| Пункт:", doc.metadata["пункт"])
    print("Текст:", doc.page_content[:200])
    print("-" * 60)

Документ: Приказ Министерства транспорта Российской Федерации от 24 января 2013 г. № 13 «Об утверждении Табеля сообщений о движении воздушных судов в Российской Федерации»
Глава: I. | Пункт: 2.
Текст: Настоящий Табель сообщений определяет: состав (объем) информации, включаемой в сообщение о представленном плане полета воз- душного судна, и правила передачи указанного сообщения; состав (объем) инфор
------------------------------------------------------------
Документ: ПОСТАНОВЛЕНИЕ от 11 марта 2010 г. N 138 'ОБ УТВЕРЖДЕНИИ ФЕДЕРАЛЬНЫХ ПРАВИЛ ИСПОЛЬЗОВАНИЯ ВОЗДУШНОГО ПРОСТРАНСТВА РОССИЙСКОЙ ФЕДЕРАЦИИ'
Глава: II. | Пункт: 75.
Текст: В воздушном пространстве устанавливаются минимальные интервалы горизонтального эшелонирования.
------------------------------------------------------------
Документ: Приказ Министерства транспорта Российской Федерации от 24 января 2013 г. № 13 «Об утверждении Табеля сообщений о движении воздушных судов в Российской Федерации»
Глава: II. | Пункт: 13.7.3.
Текст